# Advent of Code 2021 - Day 6

In [67]:
import java.io.File
import java.util.Scanner

val regex = Regex("""(\d+)""")

val fishes: List<Int> = Scanner(File("Day6.input.txt"))
    .apply { useDelimiter(",") }
    .use { scanner ->
        generateSequence { if(scanner.hasNext()) scanner.next() else null }
            .map(String::toInt)
            .toList()
    }

## Part 1

Each fish in `fishes` represents the number of days until it creates a new fish. When it reaches 0, its timer resets to 6 and a new fish with a timer of 8 is created. Note that 0 is included as a valid timer and a newly added fish does not count down until the next day. How many fish exist after 80 days?

In [68]:
fun List<Int>.decrement(): List<Int> = map { if (it == 0) 6 else it - 1 }

fun List<Int>.spawn(): List<Int> = fold(toMutableList()) { acc, it -> if (it == 0) acc.apply { add(9) } else acc }

fun List<Int>.simulateDay(): List<Int> = spawn().decrement()

tailrec fun List<Int>.simulate(days: Int): List<Int> = if (days != 0) simulateDay().simulate(days - 1) else this

fishes.simulate(80).size

354564

### Notes

This approach takes the problem statement pretty literally, by "simulating" each fish. Each fish decrements its internal counter, and new fish are added when appropriate.

## Part 2

Same as party 1, but how many fish exist after 256 days?

In [69]:
import java.util.Collections

fun MutableList<Long>.simulateDay(): MutableList<Long> = apply {
    this[7] = this[7] + this[0]
    Collections.rotate(this, -1)
}

fun List<Int>.initialize(): MutableList<Long> {
    val counts: MutableList<Long> = (0..8).map { 0L }.toMutableList()
    forEach { counts[it] = counts[it] + 1 }
    return counts
}

tailrec fun MutableList<Long>.numberOfFishesAfter(days: Int): MutableList<Long> = 
    if (days != 0) simulateDay().numberOfFishesAfter(days - 1) else this

fishes.initialize().numberOfFishesAfter(256).sum()

1609058859115

### Notes

The solution in part 1 quickly grows too large, so a different approach is needed. Instead, we can notice that each fish's "spot" in the list is irrelevant. What's important is the number of fishes for each possible "days left." For instance, 5 fish with 3 days left will always create 5 fish with 2 days left at the end of the day so we can stop simulating each fish. Now we can notice that the current number of fishes for each day is always the previous day's number at day + 1. This is basically a "rotation": [3,2,4] -> [2,4,3]. To model the spawning, we can "insert" day 0's fishes into the 6th day.